In [1]:
%pwd

'/Users/matt/Documents/deeplearning/research'

In [2]:
import os
os.chdir('/Users/matt/Documents/deeplearning')
%pwd


'/Users/matt/Documents/deeplearning'

In [3]:
import tensorflow as tf


2023-10-17 16:27:04.792438: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [33]:
model=tf.keras.models.load_model("artifacts/training/model.h5")


In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
  path_of_model: Path
  validation_data: Path
  all_params: dict
  params_batch_size:int
  params_image_size:list

In [42]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [44]:
class ConfigurationManager:
  def __init__(self,config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
    self.config = read_yaml(config_filepath)
    self.params = read_yaml(params_filepath)
    create_directories([self.config.artifacts_root])
    
  def get_evaluation_config(self) ->EvaluationConfig:
    eva_config=EvaluationConfig(
    path_of_model=Path("artifacts/training/model.h5"),
    validation_data=Path("artifacts/data_ingestion/Chicken-fecal-images/validation/"),
    all_params=self.params,
    params_batch_size=self.params.BATCH_SIZE,
    params_image_size=self.params.IMAGE_SIZE
    )
    return eva_config
  

In [48]:
class Evaluation:
  def __init__(self,config=EvaluationConfig):
    self.config = config
    
  def _valid_generator(self):
    
    datagenerator_kwargs = dict(
        rescale = 1./255,
        # validation_split=0.20  #  validation_split: 浮点数。Float. 保留用于验证的图像的比例（严格在0和1之间）。  
    )

    dataflow_kwargs = dict(
        target_size=self.config.params_image_size[:-1],
        batch_size=self.config.params_batch_size,
        interpolation="bilinear"
    )
    
    valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
        **datagenerator_kwargs
    )

    self.valid_generator = valid_datagenerator.flow_from_directory(
        directory=self.config.validation_data,
        shuffle=False,
        **dataflow_kwargs
    )
    
  @staticmethod
  def load_model(path:Path)->tf.keras.Model:
    return tf.keras.models.load_model(path)
  
  def evaluation(self):
    self.model=self.load_model(self.config.path_of_model)
    self._valid_generator()
    self.score=self.model.evaluate(self.valid_generator)
    
  def save_model_evaluation_score(self):
    score={"loss":self.score[0],"accuracy":self.score[1]}
    save_json(path=Path("scores.json"),data=score)

In [49]:
try:
  config_manmager=ConfigurationManager()
  evaluation_config=config_manmager.get_evaluation_config()
  evaluation=Evaluation(evaluation_config)
  evaluation.evaluation()
  evaluation.save_model_evaluation_score()
except Exception as e:
  raise e

[2023-10-17 17:34:15,869: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-10-17 17:34:15,902: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-17 17:34:15,915: INFO: common: created directory at: artifacts]
Found 98 images belonging to 2 classes.
7/7 [==============================] - 31s 4s/step - loss: 18.1792 - accuracy: 0.5000
[2023-10-17 17:34:49,584: INFO: common: json file saved at: scores.json]


In [21]:
def my_function(name, age, height, weight):
  print(name, age, height, weight)
  
adict=dict(
  name="matt",
  age=23,
  height=1.80,
  weight=70.00
)

  
my_function(**adict)
my_function(name="li", age=89, height=90, weight=68)

matt 23 1.8 70.0
li 89 90 68
